In [ ]:
# ⚙️ VERIFICAR Y PREPARAR ENTORNO
import os

print("📍 Directorio actual:", os.getcwd())

# Si estamos en Google Colab
if '/content' in os.getcwd():
    # Volver a /content primero
    os.chdir('/content')
    print("📦 Preparando entorno limpio...")
    
    # Limpiar cualquier carpeta anterior
    !rm -rf Botardo
    
    # Clonar repo
    print("📥 Clonando repositorio...")
    !git clone https://github.com/TobiasActis/Botardo.git
    
    # Entrar a la carpeta
    os.chdir('/content/Botardo')
    print("✅ Directorio configurado:", os.getcwd())
    print("📂 Archivos disponibles:")
    !ls -1
else:
    print("✅ Ejecutando localmente")

In [ ]:
# ⚙️ INSTALAR DEPENDENCIAS
print("📚 Instalando paquetes...")
!pip install -q pandas numpy matplotlib loguru ta requests
print("✅ Dependencias instaladas")

In [ ]:
# ⚙️ DESCARGAR DATOS DE BINANCE
import os

data_file = 'data/BTCUSDT_1m_2024-01-01_to_now.csv'

if os.path.exists(data_file):
    print(f"✅ Datos ya existen: {data_file}")
    # Mostrar info del archivo
    size_mb = os.path.getsize(data_file) / (1024*1024)
    print(f"📊 Tamaño: {size_mb:.1f} MB")
else:
    print("📊 Descargando datos de Binance...")
    !python download_btc_futures_1m.py
    print("✅ Datos descargados!")

In [ ]:
# 🚀 EJECUTAR BACKTEST COMPLETO 2024-2025
print("🚀 Iniciando backtest de 2 años...")
print("⏱️ Esto puede tardar 2-3 minutos\n")

!python backtest_superbot.py \
  --data_1m "data/BTCUSDT_1m_2024-01-01_to_now.csv" \
  --initial_capital 500 \
  --risk_per_trade 0.10 \
  --leverage 50 \
  --po3_min_rr 3.5 \
  --smc_standalone 8 \
  --start "2024-01-01" \
  --end "2025-12-09"

print("\n✅ Backtest completado!")

In [ ]:
# 📊 MOSTRAR GRÁFICO DE EQUITY
from IPython.display import Image, display
import os

if os.path.exists('superbot_backtest.png'):
    print("\n📈 GRÁFICO DE EQUITY CURVE:")
    display(Image('superbot_backtest.png', width=900))
else:
    print("⚠️ No se encontró el gráfico - El backtest pudo no generar trades")

In [ ]:
# 📋 ANALIZAR TRADES EN DETALLE
import pandas as pd
import os

if not os.path.exists('superbot_backtest_trades.csv'):
    print("❌ No se generó archivo de trades - probablemente no hubo señales")
    print("\n💡 Prueba ajustar parámetros:")
    print("   - Bajar --po3_min_rr de 3.5 a 3.0")
    print("   - Bajar --smc_standalone de 8 a 6")
else:
    # Cargar trades
    trades = pd.read_csv('superbot_backtest_trades.csv')
    
    print(f"\n📊 RESUMEN DE TRADES:")
    print(f"="*60)
    print(f"Total trades: {len(trades)}")
    wins = trades[trades['pnl'] > 0]
    losses = trades[trades['pnl'] < 0]
    print(f"Wins: {len(wins)} ({len(wins)/len(trades)*100:.1f}%)")
    print(f"Losses: {len(losses)} ({len(losses)/len(trades)*100:.1f}%)")
    
    print(f"\n💰 RESULTADOS FINANCIEROS:")
    print(f"="*60)
    print(f"Total P&L: ${trades['pnl'].sum():.2f}")
    print(f"ROI: {trades['pnl'].sum()/500*100:.2f}%")
    if len(wins) > 0:
        print(f"Average Win: ${wins['pnl'].mean():.2f} ({wins['pnl_pct'].mean():.2f}%)")
    if len(losses) > 0:
        print(f"Average Loss: ${losses['pnl'].mean():.2f} ({losses['pnl_pct'].mean():.2f}%)")
    print(f"Best Trade: ${trades['pnl'].max():.2f} ({trades['pnl_pct'].max():.2f}%)")
    print(f"Worst Trade: ${trades['pnl'].min():.2f} ({trades['pnl_pct'].min():.2f}%)")
    
    # BUSCAR TRADES CON SL INVERTIDO (BUG)
    print(f"\n🔍 VERIFICACIÓN DE BUG (SL invertido):")
    print(f"="*60)
    
    # Trades LONG con SL > entry
    long_bad = trades[(trades['side'] == 'LONG') & (trades['stop_loss'] > trades['entry_price'])]
    # Trades SHORT con SL < entry  
    short_bad = trades[(trades['side'] == 'SHORT') & (trades['stop_loss'] < trades['entry_price'])]
    
    if len(long_bad) > 0 or len(short_bad) > 0:
        print(f"❌ ENCONTRADOS {len(long_bad) + len(short_bad)} TRADES CON SL INVERTIDO:")
        if len(long_bad) > 0:
            print(f"\n   LONG con SL > entry: {len(long_bad)} trades")
            print(long_bad[['timestamp', 'entry_price', 'stop_loss', 'pnl', 'exit_reason']].head())
        if len(short_bad) > 0:
            print(f"\n   SHORT con SL < entry: {len(short_bad)} trades")
            print(short_bad[['timestamp', 'entry_price', 'stop_loss', 'pnl', 'exit_reason']].head())
    else:
        print("✅ TODOS LOS TRADES TIENEN SL CORRECTO")
        print("   - LONGs: SL < entry")
        print("   - SHORTs: SL > entry")
    
    print(f"\n📋 ÚLTIMOS 10 TRADES:")
    print("="*60)
    display_cols = ['timestamp', 'side', 'entry_price', 'stop_loss', 'take_profit', 'exit_price', 'pnl', 'pnl_pct', 'exit_reason']
    print(trades[display_cols].tail(10).to_string())

In [ ]:
# 📅 ANÁLISIS MENSUAL
import pandas as pd
import os

if os.path.exists('superbot_backtest_trades.csv'):
    trades = pd.read_csv('superbot_backtest_trades.csv')
    trades['timestamp'] = pd.to_datetime(trades['timestamp'])
    trades['month'] = trades['timestamp'].dt.to_period('M')
    
    monthly = trades.groupby('month').agg({
        'pnl': ['sum', 'count'],
        'pnl_pct': 'mean'
    }).round(2)
    
    monthly.columns = ['P&L Total ($)', 'Trades', 'Avg %']
    
    print("\n📅 PERFORMANCE MENSUAL:")
    print("="*60)
    print(monthly.to_string())
    
    # Calcular meses ganadores vs perdedores
    winning_months = len(monthly[monthly['P&L Total ($)'] > 0])
    losing_months = len(monthly[monthly['P&L Total ($)'] < 0])
    print(f"\n✅ Meses ganadores: {winning_months}")
    print(f"❌ Meses perdedores: {losing_months}")
    print(f"📊 Consistencia: {winning_months/(winning_months+losing_months)*100:.1f}%")

In [ ]:
# 💾 DESCARGAR RESULTADOS A TU PC
try:
    from google.colab import files
    
    print("💾 Descargando archivos a tu PC...")
    
    if os.path.exists('superbot_backtest.png'):
        files.download('superbot_backtest.png')
        print("✅ Gráfico descargado")
    
    if os.path.exists('superbot_backtest_trades.csv'):
        files.download('superbot_backtest_trades.csv')
        print("✅ CSV de trades descargado")
    
    print("\n✅ Todos los archivos descargados!")
except ImportError:
    print("ℹ️ No estás en Google Colab - los archivos ya están en tu PC")

---

## ⚙️ CAMBIAR PARÁMETROS

Edita la celda del **PASO 4** para ajustar:

```bash
--initial_capital 500     # Capital inicial ($)
--risk_per_trade 0.10     # 10% de riesgo por trade
--leverage 50             # Leverage máximo (5-75)
--po3_min_rr 3.5          # RR mínimo para entrar (3.0-5.0)
--smc_standalone 8        # Confluence mínima SMC (6-10)
--start "2024-01-01"      # Fecha inicio
--end "2025-12-09"        # Fecha fin
```

**Recomendaciones:**
- Si hay pocos trades: Bajar `po3_min_rr` a 3.0 o `smc_standalone` a 6
- Si hay muchas pérdidas: Subir `po3_min_rr` a 4.0 o `smc_standalone` a 9
- Si drawdown >30%: Bajar `leverage` a 30 o `risk_per_trade` a 0.05